# CuPy

现在我们已经用 Numba 探索了一些底层 GPU API，让我们转向使用 [CuPy](https://cupy.dev/) 处理一些高级数组功能。

CuPy 有来自包括 NVIDIA 在内的多个组织的维护者。CuPy 实现了熟悉的 NumPy API，但后端是用 CUDA C++ 编写的。这使得已经熟悉 NumPy 的人只需更换一个导入语句就可以快速获得 GPU 加速。

In [ ]:
import numpy as np
import cupy as cp
cp.cuda.Stream.null.synchronize()

让我们来看看这篇博文中的一些简单示例：https://towardsdatascience.com/heres-how-to-use-cupy-to-make-numpy-700x-faster-4b920dda1f56

## 创建数组

首先让我们在 CPU 和 GPU 上分别创建一个 `2GB` 的数组，并比较这需要多长时间。

In [ ]:
%%timeit -r 1 -n 10
global x_cpu
x_cpu = np.ones((1000, 500, 500))

In [ ]:
%%timeit -n 10
global x_gpu
x_gpu = cp.ones((1000, 500, 500))

cp.cuda.Stream.null.synchronize()

_注意，为了让我们的计时公平，这里需要显式调用 `cp.cuda.Stream.null.synchronize()`。默认情况下，cupy 会并发运行 GPU 代码，函数会在 GPU 完成之前就返回。调用 `synchronize()` 会让我们等待 GPU 完成后再返回。_

我们可以看到，在 GPU 上创建这个数组比在 CPU 上快得多，但这次我们的代码看起来完全一样。我们不需要担心内核、线程、块或任何这些东西。

## 基本操作

接下来让我们看看对数组进行一些数学运算。我们可以从将数组中的每个值乘以 `5` 开始。

In [ ]:
%%time
x_cpu *= 5

In [ ]:
%%time
x_gpu *= 5

cp.cuda.Stream.null.synchronize()

GPU 再次完成得更快，但代码保持不变。

现在让我们连续执行几个操作，这在我们的 Numba 示例中如果没有显式内存管理会受到内存传输时间的影响。

In [ ]:
%%time
x_cpu *= 5
x_cpu *= x_cpu
x_cpu += x_cpu

In [ ]:
%%time
x_gpu *= 5
x_gpu *= x_gpu
x_gpu += x_gpu

cp.cuda.Stream.null.synchronize()

我们可以看到，即使没有显式管理内存，GPU 运行速度也快得多。这是因为 CuPy 在后台为我们透明地处理了所有这些。

## 更复杂的操作

现在我们已经尝试了一些运算符，让我们深入研究一些 NumPy 函数。让我们比较一下在一个稍小的数据数组上运行奇异值分解。

In [ ]:
%%time
x_cpu = np.random.random((1000, 1000))
u, s, v = np.linalg.svd(x_cpu)

In [ ]:
%%time
x_gpu = cp.random.random((1000, 1000))
u, s, v = cp.linalg.svd(x_gpu)

正如我们所见，使用完全相同的 API，GPU 再次优于 CPU。

这里还有一点有趣的是，NumPy 可以智能地分派这样的函数调用。在上面的示例中，我们调用了 `cp.linalg.svd`，但我们也可以调用 `np.linalg.svd` 并传入我们的 GPU 数组。NumPy 会检查输入并代表我们调用 `cp.linalg.svd`。这使得在代码中引入 `cupy` 变得更加容易，只需要最少的更改。

In [ ]:
%%time
x_gpu = cp.random.random((1000, 1000))
u, s, v = np.linalg.svd(x_gpu)  # 注意这里使用的是 `np`

cp.cuda.Stream.null.synchronize()

## 设备

CuPy 有当前设备的概念，它是数组分配、操作、计算等默认发生的 GPU 设备。假设当前设备的 ID 是 `0`。在这种情况下，以下代码会在 GPU 0 上创建一个数组 `x_on_gpu0`。

In [ ]:
with cp.cuda.Device(0):
   x_on_gpu0 = cp.random.random((100000, 1000))

x_on_gpu0.device

通常，CuPy 函数期望数组与当前设备在同一设备上。根据硬件配置，传递存储在非当前设备上的数组可能会工作，但通常不建议这样做，因为它可能不会有好的性能。